# Measure the ellipticity of cucurbituril hosts

This notebook runs the following pipeline to measure host ellipticity. 

![pipeline image](https://github.com/harmslab/ElliptiC/blob/main/images/pipeline_image.png)

For more details, please see the documentation:

https://github.com/harmslab/ElliptiC

A manuscript describing this approach is forthcoming.

In [ ]:
#@title Set up environment.

#@markdown This cell configures the computing environment to run ElliptiCbn. Run
#@markdown this cell before uploading an XYZ file.

#@markdown To run the cell, click the "Play" button to the left.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e:
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    %pip install git+https://github.com/harmsm/ElliptiC
    #%pip install -q ElliptiCBn

# ------------------------------------------------------------------------------
# Imports

import ElliptiCBn as ec

import numpy as np
import pandas as pd

import shutil
import os
import inspect
import re

# Set these here in case the user does not run the cell below.
bond_dist = 2.5
aspect_ratio_filter = 3
oxygen_dist_cutoff = 2.9
min_num_carbons = 10
max_num_carbons = 20
min_cycle_cc_bond_length = 1.3
max_cycle_cc_bond_length = 1.7
output_dir = "."
overwrite = False

# Note that we're getting get_macrocycles doc string because this has the
# relevant user-settable parameters for a colab-level user. If someone wants
# to set parameters to ec.run_all, they can run help on that within a 
# python environment. 
print("\nParameter descriptions\n")
doc_string = dict(inspect.getmembers(ec.get_macrocycles))["__doc__"]
doc_string = doc_string.split("Returns")[0]

lines = doc_string.split("\n")

print("\n".join(lines[7:]))

In [ ]:
#@title Set calculation parameters

#@markdown The default parameters should work for most calculations, but you can
#@markdown edit them if necessary to pull out the correct macrocycles. After
#@markdown setting the parameters, press the "Play" button on the left before
#@markdown proceeding. Parameter descriptions are above.

bond_dist = 2.5 #@param
aspect_ratio_filter = 3 #@param
oxygen_dist_cutoff = 2.9 #@param
min_num_carbons = 10 #@param
max_num_carbons = 20 #@param
min_cycle_cc_bond_length = 1.3 #@param
max_cycle_cc_bond_length = 1.7 #@param


In [ ]:
#@title Analyze a file.

#@markdown You can either upload a single .xyz file OR a .zip file containing
#@markdown multiple .xyz files. 

#@markdown Press the "Play" button do upload the file. 


if RUNNING_IN_COLAB:

    from google.colab import files
    f = files.upload()
    filename = list(f.keys())[0]

    out_base = f"{filename}_out"
    if not os.path.isdir(out_base):
        os.mkdir(out_base)

    if filename.split(".")[-1].lower() == "zip":
        
        input_dir = ".".join(filename.split(".")[:-1])
        
        shutil.unpack_archive(filename,input_dir)
        
        match_pattern = re.compile("\.xyz",re.IGNORECASE)
        file_input = os.listdir(input_dir)
        file_input = [os.path.join(input_dir,f)
                      for f in file_input if match_pattern.search(f)]
        file_input.sort()
        
        output_dir = f"{input_dir}_output"
        
    else:
        
        file_input = [filename]
        output_dir = f"{input_dir}_output"

        
fig = ec.run_all(file_input,
                 bond_dist=bond_dist,
                 aspect_ratio_filter=aspect_ratio_filter,
                 oxygen_dist_cutoff=oxygen_dist_cutoff,
                 min_num_carbons=min_num_carbons,
                 max_num_carbons=max_num_carbons,
                 min_cycle_cc_bond_length=min_cycle_cc_bond_length,
                 max_cycle_cc_bond_length=max_cycle_cc_bond_length,
                 output_dir=output_dir,
                 overwrite=overwrite)

if fig is not None:
    fig.show()
    

In [ ]:
#@title Download results.

#@markdown To download the results, press the "Play" button on the left.

if RUNNING_IN_COLAB:

    shutil.make_archive(base_name=output_dir,
                        format="zip",
                        base_dir=output_dir)
    print(f"Put results in {output_dir}.zip")

    files.download(f"{output_dir}.zip")